This program will create a GIF of an infectious disease spreading through a population according to the Reed-Frost model.

The Reed-Frost model is a simple epidemiologic model in which each individual in a population takes one of three possible states: Susceptible (S), Infected (I), or Recovered (R). At each time step, every infected (I) individual has some probability p of coming into contact with each susceptible (S) person, independently of the others. If a susceptible individual comes into contact with at least one infected individual, they will become infected for the next time step and recovered (R) the time step after that. As such, each infected individual is only infected for one time step and cannot be re-infected after they reach "recovered" status. 

This program takes three parameters: 
- size: the total number of people in the population
- I0: the initial number of infected individuals (the rest of the population will start as susceptible)
- p: the probability of contact between each infected and susceptible individual

In the GIF, susceptible individuals will appear as green dots, infected individuals will appear as red dots, and recovered individuals will appear as gray dots.

Here's the simulation code:

First, you'll need the R packages 'igraph' and 'animation':

In [ ]:
library(igraph)
library(animation)

The animation package also requires having a program called GraphicsMagick installed on your computer. Once you have it installed, tell R the file pathway (replace "path-to-file" with the actual file pathway):

ani.options("convert"="path-to-file")

In [ ]:
ani.options("convert"="C:/Users/jamie/Downloads/GraphicsMagick-1.3.35-Q16-win64-dll") #pathway on my computer

Create the function for the simulation:

In [ ]:
## Creates a GIF of a disease spreading through a population according to the 
## Reed-Frost Susceptible-Infected-Recovered (SIR) Model
#
## size = population size
## I0 = initial number of infected individuals
## p = probability of contact between each infected and susceptible person at each time step
rfanimation <- function(size, I0, p) {
  g1 <- graph(edges=c(), n=size, directed=F) #create a graph with size vertices, no edges
  Status <- c() #vector representing the status of each person in the population
  for (i in 1:(I0)) { #set the first I0 people to "infected"
    Status <- c(Status, "I")
  }
  for (i in 1:(size - I0)) { #set the rest of the population to "susceptible"
    Status <- c(Status, "S")
  }
  V(g1)$status <- Status # assign the elements in the "Status" vector to the nodes in the graph
  
  #begin the animation
  saveGIF({
    #plot the initial graph
    ## Susceptible "S" = green
    ## Infected "I" = red
    ## Recovered "R" = gray
    plot(g1, vertex.color=c("green", "red", "gray")[1+(V(g1)$status=="I")+(2*(V(g1)$status=="R"))], 
         vertex.size=6, vertex.label=NA, 
         main=paste("Reed-Frost Simulation, population size =", size, ", p =", p))
    legend(1, 1, legend=c("S", "I", "R"), pch=19, col=c("green", "red", "gray"))
      
    newStatus <- Status #vector for the status of the nodes at the new time step
    
    while ("I" %in% Status) { #while there are infected people in the population  
      for (i in 1:length(Status)) { #for each node in the population
        if (Status[i] == "I") { #if they are infected
          newStatus[i] <- "R" #set their new status to recovered          
          for (j in 1:length(Status)) { #traverse the population again
            if (Status[j] == "S") { #if they are susceptible
              u <- runif(1) #create a u ~ Unif(0, 1) random variable
              if (u < p) { #if u is less than the p specified in the parameters
                g1 <- g1 + edge(i,j) #draw an edge between the infected node and the susceptible node
                if (newStatus[j] != "I") { #if node j hasn't already been infected this time step
                  newStatus[j] <- "I" #set its new status to infected
                }
              }
            }
          }
        }
      }
  
      Status <- newStatus #set status to new status
      V(g1)$status <- Status #assign the new status to each node in the graph
      
      # plot the new time step
      plot(g1, vertex.color=c("green", "red", "gray")[1+(V(g1)$status=="I")+(2*(V(g1)$status=="R"))], 
       vertex.size=6, vertex.label=NA,
       main=paste("Reed-Frost Simulation, population size =", size, ", p =", p))
      legend(1, 1, legend=c("S", "I", "R"), pch=19, col=c("green", "red", "gray"))
    }
  
  }, interval=2, movie.name="rf_animation.gif")
}

Finally, run the simulation. For example, try a population of 150 people with 1 initial infection and p=0.015:

In [ ]:
rfanimation(150, 1, 0.015)